## Project:  "Free Azure VMs are Hard to Find!!!"
### Author:  Dale Cosh | dmcosh@gmail.com | Database Administrator and aspiring Data Engineer
### Issue: 
**Microsoft offers 12 months of Azure with a number of free resources, including certain types of Virtual Machines.  However, Azure is very popular and free Azure even more so ...which makes the free VMs almost impossible to get at times.  The ever present "NotAvailableForSubscription" is maddening and randomly picking different regions is tedious.**
### Solution:
**Utilize Microsoft's REST API to pull listing of locations where free VMs are currently available so we know what regions to target when creating a VM in Azure portal or CLI.**

Created:  2026-01-18

Modified: 2026-01-28 refactored Polars code to utilize method chaining - DCosh

Prerequisites: 
- pip install dotenv
- pip install requests
- pip install polars
- pip install azure-identity
- Create an Azure account
- Create Azure service principal (App registration) and environment variables AZURE_CLIENT_ID, AZURE_TENANT_ID, AZURE_CLIENT_SECRET,AZURE_SUBSCRIPTION_ID.  For more info: https://learn.microsoft.com/en-us/azure/developer/python/sdk/authentication/overview#authentication-during-local-development


In [9]:
from dotenv import load_dotenv 
import os
import requests
import polars as pl
from azure.identity import DefaultAzureCredential

# import logging
# logging.basicConfig(level=logging.DEBUG)

load_dotenv() # Loads variables from .env file
azure_client_id         = os.getenv("AZURE_CLIENT_ID")
azure_client_secret     = os.getenv("AZURE_CLIENT_SECRET")
azure_tenant_id         = os.getenv("AZURE_TENANT_ID")
azure_subscription_id   = os.getenv("AZURE_SUBSCRIPTION_ID")

# these are the free eligible VM types as of script creation (x86 only, update if looking for ARM architecture)
free_eligible_skus = ["Standard_B1s", "Standard_B2ats_v2"] 
sku_type = "virtualMachines"

In [10]:
# Get auth and token for Azure REST API calls
try:
    scope = "https://management.azure.com/.default" # app uri plus /.default
    azure_cred = DefaultAzureCredential()
    access_token = azure_cred.get_token(scope).token
except Exception as e:
    print(f"Authentication failed: {e}")

headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}


In [11]:
# Get locations list so Regional Display Name can be in final output
url = "https://management.azure.com/subscriptions/" + azure_subscription_id + "/locations?api-version=2025-04-01"
response = requests.get(url, headers=headers)

if response.status_code == 200:
    # print(help(response))
    response_loc = response.json() # Retrieves data as list of Python dictionaries
else:
    print(f"Error: {response.status_code} - {response.text}")


In [12]:
# Get Resource SKUs list for availability of VMs
url = "https://management.azure.com/subscriptions/" + azure_subscription_id + "/providers/Microsoft.Compute/skus?api-version=2025-04-01"
response = requests.get(url, headers=headers)

if response.status_code == 200:
    response_sku = response.json() # Retrieves data as list of Python dictionaries
else:
    print(f"Error: {response.status_code} - {response.text}")


In [13]:
# Get registered providers list for your subscription (this will eliminate locations that aren't actually available to you)
url = "https://management.azure.com/subscriptions/" + azure_subscription_id + "/providers?api-version=2025-04-01"
response = requests.get(url, headers=headers)

if response.status_code == 200:
    response_prov = response.json() # Retrieves data as list of Python dictionaries
else:
    print(f"Error: {response.status_code} - {response.text}")


In [14]:
# Pull response files into Polars DataFrames and massage data to make it useful
# refactored code in this cell to utilize method chaining 2026.01.28 - DCosh

# Locations
# pldf_locations = pl.from_dicts(response_loc["value"]) # replaced 2026.01.28 - DCosh
# pldf_locations = pldf_locations.unnest("metadata") # replaced 2026.01.28 - DCosh
pldf_locations = ( 
    pl.from_dicts(response_loc["value"])
    .unnest("metadata")
)

# SKUs
# replaced 2026.01.28 - DCosh
# pldf_skus = pl.from_dicts(response_sku["value"], infer_schema_length=None) # scan all rows because schema varies wildly
# pldf_skus = pldf_skus.filter( (pl.col("resourceType") == "virtualMachines") 
                            #  Select the 'eligible for free services' types of VMs only
                            #  & (pl.col("name").is_in(free_eligible_skus)) 
# )
pldf_skus = (
    pl.from_dicts(response_sku["value"], infer_schema_length=None)
    .filter(
        (pl.col("resourceType") == sku_type) &
        (pl.col("name").is_in(free_eligible_skus))
    )
)

# Split restrictions into separate DataFrame and unpack all levels
# replaced 2026.01.28 - DCosh
# pldf_restrictions = pldf_skus["name","restrictions"]
# pldf_restrictions = pldf_restrictions.explode("restrictions")
# pldf_restrictions = pldf_restrictions.unnest("restrictions", separator="_")
# pldf_restrictions = pldf_restrictions.drop("restrictions_values")
# pldf_restrictions = pldf_restrictions.unnest("restrictions_restrictionInfo", separator="_")
# pldf_restrictions = pldf_restrictions.explode("restrictions_restrictionInfo_zones")
# pldf_restrictions = pldf_restrictions.explode("restrictions_restrictionInfo_locations")
# pldf_restrictions = pldf_restrictions.drop_nulls(subset=["restrictions_reasonCode"])

pldf_restrictions = (
    pldf_skus
    .select("name", "restrictions")
    .explode("restrictions")
    .unnest("restrictions", separator="_")
    .drop("restrictions_values")
    .unnest("restrictions_restrictionInfo", separator="_")
    .explode("restrictions_restrictionInfo_zones")
    .explode("restrictions_restrictionInfo_locations")
    .drop_nulls(subset=["restrictions_reasonCode"])
)

# Go back to SKUs, drop unecessary columns and unpack remaining
# replaced 2026.01.28 - DCosh
# pldf_skus = pldf_skus.drop(["locations", "capabilities", "restrictions", "tier", "size", "family"])
# pldf_skus = pldf_skus.explode("locationInfo")
# pldf_skus = pldf_skus.unnest("locationInfo", separator="_")
# pldf_skus = pldf_skus.drop(["locationInfo_zoneDetails"])
# pldf_skus = pldf_skus.explode("locationInfo_zones")

pldf_skus = (
    pldf_skus
    .drop(["locations", "capabilities", "restrictions", "tier", "size", "family"])
    .explode("locationInfo")
    .unnest("locationInfo", separator="_")
    .drop("locationInfo_zoneDetails")
    .explode("locationInfo_zones")
)

# Providers
# replaced 2026.01.28 - DCosh
# pldf_providers = pl.from_dicts(response_prov["value"])
# pldf_providers = pldf_providers.filter( (pl.col("namespace") == "Microsoft.Compute") & (pl.col("registrationState") == "Registered") )
# pldf_providers = pldf_providers.explode("resourceTypes")
# pldf_providers = pl.DataFrame(pldf_providers["resourceTypes"])
# pldf_providers = pldf_providers.unnest("resourceTypes")
# pldf_providers = pldf_providers.filter(pl.col("resourceType") == "virtualMachines")
# pldf_providers = pldf_providers.explode("locations")

pldf_providers = (
    pl.from_dicts(response_prov["value"])
    .filter(
        (pl.col("namespace") == "Microsoft.Compute") & 
        (pl.col("registrationState") == "Registered")
    )
    .select("resourceTypes")
    .explode("resourceTypes")
    .unnest("resourceTypes")
    .filter(pl.col("resourceType") == sku_type)
    .explode("locations")
)

In [15]:
# Join dataframes together using Polars SQL Engine, because SQL is easier for me ;)
with pl.SQLContext() as ctx:
    ctx.register_many(s=pldf_skus, r=pldf_restrictions, l=pldf_locations, p=pldf_providers).tables()
    pldf_final = ctx.execute("""
                             SELECT 
                                s.name AS vm,
                                l.regionalDisplayName AS region,
                                l.name as name,
                                COALESCE(s.locationInfo_zones, '') AS zone
                             FROM s
                             INNER JOIN l ON lower(s.locationInfo_location) = l.name
                             INNER JOIN p ON l.displayName = p.locations
                             -- Polars doesn't support NOT EXISTS statement so writing this as left join with where NULL clause instead
                             LEFT JOIN  r ON s.name = r.name
                                         AND s.locationInfo_location = r.restrictions_restrictionInfo_locations
                                         AND COALESCE(s.locationInfo_zones, 999999) = COALESCE(r.restrictions_restrictionInfo_zones, 999999) 
                             WHERE r.restrictions_reasonCode IS NULL 
                             ORDER BY vm, region, zone
                             ;"""
                             ,eager=True
                             )


In [16]:
# FINALLY, output list of VMs and Regions where they are available... 
with pl.Config(tbl_rows=-1, tbl_width_chars=100, fmt_str_lengths=80):
    print(pldf_final.select(pl.col("vm").alias("VM Size"), 
                            pl.col("region").alias("Region (portal)"), 
                            pl.col("name").alias("Location (CLI)"),
                            pl.col("zone").alias("Zone")
                            )
          )


shape: (73, 4)
┌───────────────────┬────────────────────────────────────┬────────────────────┬──────┐
│ VM Size           ┆ Region (portal)                    ┆ Location (CLI)     ┆ Zone │
│ ---               ┆ ---                                ┆ ---                ┆ ---  │
│ str               ┆ str                                ┆ str                ┆ str  │
╞═══════════════════╪════════════════════════════════════╪════════════════════╪══════╡
│ Standard_B2ats_v2 ┆ (Africa) South Africa North        ┆ southafricanorth   ┆ 1    │
│ Standard_B2ats_v2 ┆ (Africa) South Africa North        ┆ southafricanorth   ┆ 2    │
│ Standard_B2ats_v2 ┆ (Africa) South Africa North        ┆ southafricanorth   ┆ 3    │
│ Standard_B2ats_v2 ┆ (Asia Pacific) Australia Central   ┆ australiacentral   ┆      │
│ Standard_B2ats_v2 ┆ (Asia Pacific) Australia East      ┆ australiaeast      ┆ 1    │
│ Standard_B2ats_v2 ┆ (Asia Pacific) Australia East      ┆ australiaeast      ┆ 2    │
│ Standard_B2ats_v2 ┆ (Asia 